In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
from keras.models import Model
import tensorflow as tf
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt

In [22]:
# Hàm chuyển đổi một tệp JSON thành định dạng đầu ra phù hợp
def process_json(json_data, folder_name, dataset_path):
    # Trích xuất thông tin từ tệp JSON
    image_path = os.path.join(dataset_path, folder_name, json_data["imagePath"])
    annotations = []

    for shape in json_data["shapes"]:
        points = shape["points"]
        x_min = min(points, key=lambda x: x[0])[0]
        x_max = max(points, key=lambda x: x[0])[0]
        y_min = min(points, key=lambda x: x[1])[1]
        y_max = max(points, key=lambda x: x[1])[1]

        width = x_max - x_min
        height = y_max - y_min

        annotations.append({
            "Label": shape["label"],
            "Bbox": [x_min, y_min, width, height]
        })

    return {
        "image_path": image_path,
        "annotations": annotations
    }

dataset_path = "/content/drive/MyDrive/Colab Notebooks/Local/Data"
# Tạo danh sách để lưu trữ dữ liệu định dạng đầu ra
formatted_data = []

for folder_name in os.listdir(dataset_path):
    json_folder = os.path.join(dataset_path, folder_name)

    # Xử lý dữ liệu từ các tệp JSON trong thư mục
    for filename in os.listdir(json_folder):
        if filename.endswith('.json'):
            file_path = os.path.join(json_folder, filename)
            with open(file_path, 'r') as json_file:
                json_data = json.load(json_file)
                formatted_data.append(process_json(json_data, folder_name, dataset_path))

from sklearn.model_selection import train_test_split

# Dữ liệu đã được chuẩn hóa và lưu trong biến formatted_data

# Tách dữ liệu thành tập huấn luyện (60%), tập xác thực (20%) và tập kiểm tra (20%)
train_data, temp_data = train_test_split(formatted_data, test_size=0.4, random_state=42)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# In số lượng mẫu trong mỗi tập
print(f"Số lượng mẫu trong tập huấn luyện: {len(train_data)}")
print(f"Số lượng mẫu trong tập xác thực: {len(validation_data)}")
print(f"Số lượng mẫu trong tập kiểm tra: {len(test_data)}")

# Kích thước đầu vào mong muốn của mô hình
input_shape = (224, 224, 3)

# Chuyển đổi dữ liệu hình ảnh và nhãn thành NumPy arrays và điều chỉnh kích thước ảnh
def load_and_resize_images(data):
    images = []
    bboxes = []
    labels = []

    for item in data:
        image = cv2.imread(item['image_path'])
        image = cv2.resize(image, (input_shape[0], input_shape[1]))

        annotation = item["annotations"]
        bbox_list = []
        label_list = []
        image_list = []

        for ann in annotation:
            # Thay đổi nhãn "Airplane" thành 1 và "Helicopter" thành 0
            label = 1 if ann['Label'] == 'Airplane' else 0
            bbox_list.append(ann['Bbox'])
            label_list.append(label)
            image_list.append(image)

        images.extend(image_list)
        bboxes.extend(bbox_list)
        labels.extend(label_list)  # Sử dụng extend() để thêm các giá trị của label_list vào labels

    images = np.array(images)
    bboxes = np.array(bboxes)
    labels = np.array(labels)

    return images, [bboxes, labels]

# Chia dữ liệu thành các biến
# Chia dữ liệu thành các biến
x_train, [y_train_bbox, y_train_labels] = load_and_resize_images(train_data)
x_validation, [y_validation_bbox, y_validation_labels] = load_and_resize_images(validation_data)
x_test, [y_test_bbox, y_test_labels] = load_and_resize_images(test_data)


Số lượng mẫu trong tập huấn luyện: 300
Số lượng mẫu trong tập xác thực: 100
Số lượng mẫu trong tập kiểm tra: 100


In [35]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,Flatten, Dense, Dropout, Input

# Xây dựng mô hình
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))

# Đầu ra cho bounding box (4 giá trị: x, y, width, height)
bbox_output = Dense(4, activation='linear', name='BBox')(model.layers[-1].output)

# Đầu ra cho label (1 giá trị xác suất)
label_output = Dense(1, activation='sigmoid', name='Label')(model.layers[-1].output)

# Tạo mô hình
model = Model(inputs=model.inputs, outputs=[bbox_output, label_output])

# Kết hợp hàm mất mát
model.compile(optimizer='adam',
              loss={'Label': 'binary_crossentropy', 'BBox': 'mean_squared_error'},
              metrics={"Label":'accuracy'})

# Kiểm tra mô hình
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 conv2d_15_input (InputLaye  [(None, 224, 224, 3)]        0         []                            
 r)                                                                                               
                                                                                                  
 conv2d_15 (Conv2D)          (None, 222, 222, 32)         896       ['conv2d_15_input[0][0]']     
                                                                                                  
 batch_normalization (Batch  (None, 222, 222, 32)         128       ['conv2d_15[0][0]']           
 Normalization)                                                                                   
                                                                                            

In [41]:
# Huấn luyện mô hình với nhãn đã được mã hóa
history = model.fit(
    x_train,
    {'BBox': y_train_bbox, 'Label': y_train_labels},
    validation_data=(x_validation, {'BBox': y_validation_bbox, 'Label': y_validation_labels}),
    epochs=20,
    batch_size=15,
    verbose=1
)


Epoch 1/20
21/21 [==============================] - 51s 2s/step - loss: 9861.3672 - BBox_loss: 9860.8506 - Label_loss: 0.5165 - Label_accuracy: 0.9468 - val_loss: 341472.8125 - val_BBox_loss: 341466.6250 - val_Label_loss: 6.1564 - val_Label_accuracy: 0.7000
Epoch 2/20
21/21 [==============================] - 51s 2s/step - loss: 9563.5381 - BBox_loss: 9563.1670 - Label_loss: 0.3718 - Label_accuracy: 0.9402 - val_loss: 324749.4062 - val_BBox_loss: 324744.8125 - val_Label_loss: 4.6176 - val_Label_accuracy: 0.7400
Epoch 3/20
21/21 [==============================] - 52s 2s/step - loss: 13076.5439 - BBox_loss: 13075.9648 - Label_loss: 0.5790 - Label_accuracy: 0.9402 - val_loss: 313578.1250 - val_BBox_loss: 313574.0938 - val_Label_loss: 4.0972 - val_Label_accuracy: 0.7900
Epoch 4/20
21/21 [==============================] - 53s 3s/step - loss: 17482.9766 - BBox_loss: 17482.3340 - Label_loss: 0.6424 - Label_accuracy: 0.9369 - val_loss: 341505.3750 - val_BBox_loss: 341501.5938 - val_Label_loss: 

In [ ]:
# Lưu model vào đường dẫn được chỉ định
model.save('Localization_Airplane_or_Helicopter1.1.keras')

In [42]:
# Đánh giá mô hình trên tập kiểm tra
test_scores = model.evaluate(
    x_test,
    {'BBox': np.array(y_test_bbox), 'Label': np.array(y_test_labels)},
    verbose=1
)

# In kết quả đánh giá
print("Test Losses:")
print("Bounding Box Loss:", test_scores[1])
print("Label Loss:", test_scores[2])
print("Accuracy:", test_scores[3])

4/4 [==============================] - 5s 1s/step - loss: 450592.9375 - BBox_loss: 450589.0000 - Label_loss: 3.9115 - Label_accuracy: 0.7885
Test Losses:
Bounding Box Loss: 450589.0
Label Loss: 3.911515951156616
Accuracy: 0.7884615659713745
